In [1]:
import transformers
import getpass
import torch
import os 
from transformers import LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig, AutoTokenizer, AutoConfig
from peft import PeftModel

In [2]:
default_path = os.getcwd()
data_path = os.path.join(default_path, '../../data')
model_path = os.path.join(default_path, '../../model')
config_path = os.path.join(default_path, '../../config')

In [3]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda', index=0)

In [4]:
generation_config = dict(
    temperature=0.8,
    # top_k=40,
    top_p=0.95,
    # do_sample=True,
    max_new_tokens=512,
    # early_stopping=True,
    # no_repeat_ngram_size=3,
    # eos_token_id=2, 
    # num_beams=1,
    # repetition_penalty=1.1,
)

In [5]:
def gen(x, config, model, tokenizer, device):
    prompt = (
        f"아래는 작업을 설명하는 명령어입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n{x}\n\n### 응답:"
    )
    len_prompt = len(prompt)
    gened = model.generate(
        **tokenizer(prompt, return_tensors="pt", return_token_type_ids=False).to(
            device
        ),
        **config
    )
    return tokenizer.decode(gened[0]) # [len_prompt:]

In [6]:
model_id = "kfkas/Llama-2-ko-7b-Chat"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(
    model_id, device_map={"": 0},torch_dtype=torch.float16, low_cpu_mem_usage=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]